In [3]:
# File: 1_data_collection.ipynb

# Import necessary libraries
import requests  # For fetching HTML content
from bs4 import BeautifulSoup  # For parsing HTML
import pandas as pd  # For structuring data
import os  # For managing file paths
import time  # For adding delays between requests

# Define seasons and generate URLs
base_url = "https://fbref.com/en/comps/9/{season}/schedule/{season}-Premier-League-Scores-and-Fixtures"
seasons = ["2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023", "2023-2024"]
urls = [base_url.format(season=season) for season in seasons]

# Print URLs to verify
print("Generated URLs:")
for url in urls:
    print(url)

Generated URLs:
https://fbref.com/en/comps/9/2018-2019/schedule/2018-2019-Premier-League-Scores-and-Fixtures
https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures
https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures
https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures
https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures
https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures
